In [2]:
import pandas as pd

In [115]:
data_full = pd.read_csv("C:/Users/rafal/Desktop/DS/Merceri price/train.tsv", sep = '\t')

In [120]:
data = data_full.sample(100000)

Używam tylko 100 000 obserwacji.

In [121]:
data.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
360135,360135,CHANEL SPF 12 FOUNDATION IN WALNUT,2,Beauty/Makeup/Face,Chanel,30.0,1,-too dark for me -used once -like second skin ...
405088,405088,T-shirts,2,Women/Tops & Blouses/T-Shirts,NaN,7.0,0,XL size Check out my other items to save on sh...
100509,100509,14K Amethyst Ring,1,Women/Jewelry/Rings,NaN,10.0,1,"Size 7 1/2. White gold filled, amethyst ring. ..."
543338,543338,karaoke singing machine,3,"Other/Musical instruments/DJ, Electronic Music...",Singing Machine,15.0,0,The singing machine doesn't come with micropho...
1286895,1286895,Adidas,3,Men/Shoes/Athletic,Adidas,12.0,0,Light weight Good condition/ lots of life left :)


In [28]:
pv = pd.pivot_table(data, values = ['price'],index = ['brand_name'], aggfunc = ['count', np.mean, np.median])

In [29]:
pv.columns = pv.columns.get_level_values(0)

In [37]:
pv_sorted = pv.sort_values(by = ['count'], ascending = False)

In [61]:
data.describe()

,train_id,item_condition_id,price,shipping
count,1.482535e+06,1.482535e+06,1.482535e+06,1.482535e+06
mean,7.412670e+05,1.907380e+00,2.673752e+01,4.472744e-01
std,4.279711e+05,9.031586e-01,3.858607e+01,4.972124e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,3.706335e+05,1.000000e+00,1.000000e+01,0.000000e+00
50%,7.412670e+05,2.000000e+00,1.700000e+01,0.000000e+00
75%,1.111900e+06,3.000000e+00,2.900000e+01,1.000000e+00
max,1.482534e+06,5.000000e+00,2.009000e+03,1.000000e+00


In [122]:
categories = data['category_name'].str.split('/', 3, expand = True)

In [123]:
categories.isna().sum()

0      456
1      456
2      456
3    99713
dtype: int64

Wyrzucam 4 poziom kategorii i uzupełniamy NA.

In [125]:
categories = categories.drop(columns = 3, axis = 1)
categories = categories.fillna(value = 'No_category')

In [126]:
categories.head()

,0,1,2
360135,Beauty,Makeup,Face
405088,Women,Tops & Blouses,T-Shirts
100509,Women,Jewelry,Rings
543338,Other,Musical instruments,"DJ, Electronic Music & Karaoke"
1286895,Men,Shoes,Athletic


In [127]:
categories.isna().sum()

0    0
1    0
2    0
dtype: int64

In [135]:
data_cats = pd.concat([data, categories], axis = 1)

In [129]:
data_cats.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,0,1,2
360135,360135,CHANEL SPF 12 FOUNDATION IN WALNUT,2,Beauty/Makeup/Face,Chanel,30.0,1,-too dark for me -used once -like second skin ...,Beauty,Makeup,Face
405088,405088,T-shirts,2,Women/Tops & Blouses/T-Shirts,NaN,7.0,0,XL size Check out my other items to save on sh...,Women,Tops & Blouses,T-Shirts
100509,100509,14K Amethyst Ring,1,Women/Jewelry/Rings,NaN,10.0,1,"Size 7 1/2. White gold filled, amethyst ring. ...",Women,Jewelry,Rings
543338,543338,karaoke singing machine,3,"Other/Musical instruments/DJ, Electronic Music...",Singing Machine,15.0,0,The singing machine doesn't come with micropho...,Other,Musical instruments,"DJ, Electronic Music & Karaoke"
1286895,1286895,Adidas,3,Men/Shoes/Athletic,Adidas,12.0,0,Light weight Good condition/ lots of life left :),Men,Shoes,Athletic


In [142]:
y = data_cats.loc[:, 'price']
data_features = data_cats.drop(columns = ['price', 'category_name'], axis = 1)

In [143]:
data_features.loc[:, 'brand_name'] = data_features.loc[:, 'brand_name'].fillna(value = 'No_brand_name')

In [146]:
data_features.head()

,train_id,name,item_condition_id,brand_name,shipping,item_description,0,1,2
360135,360135,CHANEL SPF 12 FOUNDATION IN WALNUT,2,Chanel,1,-too dark for me -used once -like second skin ...,Beauty,Makeup,Face
405088,405088,T-shirts,2,No_brand_name,0,XL size Check out my other items to save on sh...,Women,Tops & Blouses,T-Shirts
100509,100509,14K Amethyst Ring,1,No_brand_name,1,"Size 7 1/2. White gold filled, amethyst ring. ...",Women,Jewelry,Rings
543338,543338,karaoke singing machine,3,Singing Machine,0,The singing machine doesn't come with micropho...,Other,Musical instruments,"DJ, Electronic Music & Karaoke"
1286895,1286895,Adidas,3,Adidas,0,Light weight Good condition/ lots of life left :),Men,Shoes,Athletic


## Preprocessing the name variable

In [156]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

AttributeError: module 'nltk' has no attribute 'data'

In [148]:

corpus = []
for i in range(0, 1000):
    name = re.sub('[^a-zA-Z]', ' ', data_features['name'][i])
    name = name.lower()
    name = name.split()
    ps = PorterStemmer()
    name = [ps.stem(word) for word in name if not word in set(stopwords.words('english'))]
    name = ' '.join(name)
    corpus.append(name)

AttributeError: module 'nltk' has no attribute 'data'

In [ ]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 1].values

In [114]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer

In [ ]:
preprocess = make_column_transformer